# This notebook get public OIFits (with VIS2 measurements) from OiDB associated to a target list for every calib_level and send it to OIFitsExplorer through SAMP

Load common modules & define some helper functions

In [7]:
import pyvo
import pandas
from datetime import date

simbadTapUrl="http://simbad.u-strasbg.fr/simbad/sim-tap/"
oidbTapUrl="http://tap.jmmc.fr/vollt/tap/"
oidbRootUrl="http://oidb.jmmc.fr"
jmdcRootUrl="http://jmdc.jmmc.fr"
jmdcCsvUrl=jmdcRootUrl+"/export_csv"

oidb   = pyvo.dal.TAPService(oidbTapUrl)
simbad = pyvo.dal.TAPService(simbadTapUrl)


def normalize(urls, rootUrl):
    """ Add root prefix for relative urls (that starts-with / """
    res=[]
    for e in urls:
        if e[0]=='/':
            e=rootUrl+e
        res.append(e)
    return res

def createCollection(urls, filename=None):    
    """ Generate an OIFitsExplorer collection file """
    file_urls= [ f"\n<file><file>{url}</file></file>" for url in urls ]
    buffer=f"<oixp:oiDataCollection xmlns:oixp='http://www.jmmc.fr/oiexplorer-data-collection/0.1'>{''.join( file_urls) }</oixp:oiDataCollection>"                                
    if filename:
        f=open(filename,"w")
        f.write(buffer)
    else:
        return buffer
    
import os
from pathlib import Path
from astropy.samp import SAMPIntegratedClient
client = SAMPIntegratedClient()

def sendAsOixp(urls):
    mycolFilename="mycol.oixp"
    urls=normalize(urls, oidbRootUrl)
    createCollection(urls, mycolFilename)
    
    if not client.is_connected:
        client.connect()
    # prepare message and broadcast it
    message = { "samp.mtype" : "fr.jmmc.oiexplorer.load.collection", "samp.params" : {"url" : Path(mycolFilename).absolute().as_uri()} }
    receiver_ids = client.notify_all(message)
    print(f"{mycolFilename} sent by samp")    
        

#### Collect JMDC's targets and prepare a Votable of uniq ids

In [8]:
from astropy.table import Table

useRemoteJmcsNames=False

if useRemoteJmcsNames:
    jmdc_csv=pandas.read_csv(jmdcCsvUrl)
    jmdc_ids=jmdc_csv["ID1"] # add  jmdc_csv["ID2"] ?
    jmdc_ids=[s for s in jmdc_ids.dropna().unique() if len(s) == len(s.encode()) ] # filter out non ascii entries 
    # TODO try to accept utf-8 in VOTABLE ?
else:
    # distinct ids extracted from the config set used to compute JSDC2-2017 polynoms
    jmdc_ids=["HD66811", "HD37742", "HD149757", "HD37128", "HD143275", "HD38771", "HD5394", "HD111123", "HD24398", "HD44743", "HD116658", "HD118716", "HD52089", "HD193924", "HD35468", "HD3360", "HD32630", "HD160762", "HD184171", "HD36113", "HD58350", "HD209952", "HD217675", "HD23302", "HD10144", "HD23630", "HD211838", "HD214923", "HD19356", "HD27742", "HD34085", "HD64705", "HD87901", "HD106625", "HD50634", "HD176437", "HD177756", "HD169022", "HD186882", "HD23629", "HD202850", "HD103287", "HD158643", "HD172167", "HD177724", "HD210418", "HD213558", "HD48915", "HD80007", "HD95418", "HD95608", "HD139006", "HD161868", "HD47105", "HD197345", "HD27962", "HD38678", "HD49908", "HD97633", "HD106591", "HD141003", "HD141795", "HD27045", "HD102647", "HD216956", "HD56537", "HD13161", "HD5448", "HD97603", "HD116842", "HD118022", "HD157792", "HD159561", "HD177196", "HD222603", "HD30780", "HD180777", "HD187642", "HD203280", "HD28024", "HD17094", "HD24712", "HD31964", "HD45348", "HD218396", "HD29391", "HD36673", "HD81937", "HD219080", "HD58946", "HD40136", "HD109085", "HD432", "HD164259", "HD49933", "HD58728", "HD203", "HD185395", "HD128167", "HD205289", "HD209369", "HD210027", "HD213306", "HD20902", "HD1671", "HD48737", "HD61421", "HD162003", "HD168151", "HD215648", "HD28406", "HD30652", "HD33564", "HD38393", "HD69897", "HD6210", "HD89449", "HD120136", "HD142860", "HD173667", "HD181420", "HD187929", "HD219623", "HD222368", "HD16765", "HD82328", "HD126660", "HD161592", "HD194093", "HD196885", "HD8890", "HD16895", "HD20367", "HD23596", "HD50554", "HD54605", "HD90839", "HD136202", "HD187637", "HD19994", "HD22484", "HD37350", "HD5015", "HD83808", "HD102870", "HD130948", "HD9826", "HD19373", "HD114710", "HD204867", "HD13974", "HD206860", "HD209458", "HD39587", "HD7047", "HD109358", "HD121370", "HD143761", "HD150680", "HD157214", "HD164975", "HD173764", "HD175726", "HD177153", "HD190406", "HD34029", "HD34411", "HD52973", "HD84441", "HD95128", "HD186408", "HD195564", "HD209750", "HR9045", "HD16141", "HD38858", "HD159181", "HD185269", "HD217014", "HD140538", "HD1461", "HD86728", "HD10697", "HD186427", "HD117176", "HD20630", "HD44974", "HD58899", "HD84810", "HD146233", "HD154633", "HD161797", "HD174694", "HD190228", "HD21019", "HD199665", "HD221115", "HD39004", "HD3546", "HD100407", "HD115617", "HD9270", "HD131156", "HD148856", "HD150997", "HD182572", "HD190360", "HD202109", "HD205435", "HD215182", "HD215665", "HD216131", "HD217107", "HD219615", "HD221345", "HD222107", "HD18925", "HD27256", "HD38529", "HD48329", "HD62345", "HD69830", "HD75732", "HD82885", "HD101501", "HD107383", "HD113226", "HD115659", "HD133208", "HD135722", "HD148387", "HD154345", "HD11977", "HD173416", "HD173701", "HD185958", "HD5395", "HD7087", "HD138905", "HD10700", "HD185351", "HD212496", "HD28307", "HD58207", "HD76294", "HD6186", "HD95689", "HD104985", "HD9408", "HD161178", "HD11964", "HD175306", "HD175775", "HD180711", "HD181276", "HD185144", "HD27371", "HD27697", "HD188310", "HD188512", "HD193495", "HD195820", "HD198149", "HD197989", "HD210702", "HD218560", "HD1367", "HD23249", "HD25604", "HD28305", "HD38525", "HD3651", "HD45410", "HD3712", "HD4128", "HD62141", "HD62509", "HD74442", "HD166", "HD100920", "HD113049", "HD8512", "HD118219", "HD123139", "HD10135", "HD142091", "HD145675", "HD10780", "HD158633", "HD165341", "HD169916", "HD176524", "HD177830", "HD180809", "HD180540", "HD189733", "HD203504", "HIP19849", "HD31767", "HR4518", "HD7924", "HD13189", "HD206952", "HD218356", "HD222404", "HD1522", "HD32518", "HD43232", "HD4502", "HD62044", "HD73108", "HD6582", "HD96833", "HD103095", "HD103605", "HD10476", "HD149661", "HD160290", "HD163770", "HD163917", "HD167042", "HD176408", "HD176411", "HD176678", "HD12929", "HD210745", "HD214376", "HD17361", "HD170693", "HD206778", "HD19787", "HD22049", "HD23319", "HD28292", "HD43185", "HD54719", "HD59686", "HD60294", "HD66141", "HD83618", "HD85503", "HD106574", "HD110014", "HD352", "HD118904", "HD137759", "HD140573", "HD143107", "HD150010", "HD150798", "HD152812", "HD161096", "HD169414", "HD180610", "HD186815", "HD17092", "HD214868", "HD36848", "HD4628", "HD82381", "HD10380", "HD13520", "HD211388", "HD219134", "HD16160", "HD17506", "HD20644", "HD31398", "HD3627", "HD64960", "HD65759", "HD81797", "HD98262", "HD102328", "HD127665", "HD9927", "HD139663", "HD152786", "HD156283", "HD158899", "HD183912", "HD12533", "HD186791", "HD192577", "HD99998", "HD165081", "HD200205", "HD219279", "HD223719", "HD42049", "HD4656", "HD69267", "HD70272", "HD84561", "HD87837", "HD9138", "HD131873", "HD131977", "HD133124", "HD136726", "HD137443", "HD137744", "HD139357", "HD200905", "HD82308", "HD201091", "HD209100", "HD216032", "HD17709", "HD23712", "HD26038", "HD29051", "HD29139", "HD32068", "HD49968", "HD61603", "HD76351", "HD113996", "HD120933", "HD133774", "HD138265", "HD151011", "HD157681", "HD159881", "HD164058", "HD175823", "HD183589", "HD192781", "HD120477", "HD196094", "HD201092", "HD79211", "HD80493", "HD285268", "HD317090", "HD168988", "HD88230", "HD213311", "HD25025", "HD38944", "HD57423", "HD61338", "HD64351", "HD75156", "HD79210", "HD89758", "GJ412A", "HD6860", "HD100029", "HD183439", "HD42475", "HD141477", "HD146051", "HD150543", "HD196777", "HD199305", "HD204724", "HD218329", "HD219215", "HD24512", "HD33793", "HD36395", "HD52554", "HD102212", "HD123934", "HD142804", "BD-214897", "IRC-20453", "HD216386", "HD216899", "HD39801", "HD91232", "HD141938", "CCDMJ16294-2626A", "HD156637", "HD161849", "HD95687", "HD206936", "HD207005", "HD217987", "HD1013", "HD1326", "HD224935", "HD36389", "HD42995", "HD5820", "HD84194", "HD86663", "GJ411", "HD96274", "HD112142", "HD112300", "HD117675", "HD119149", "HD144690", "GJ649", "HD12479", "HR7536", "IRC+20155", "HD175929", "HD217906", "HD285968", "HD190658", "HD97671", "HD207757", "HD209950", "HD219576", "HD224427", "HD30959", "HD40239", "HD44478", "HD59086", "HD66875", "HD76827", "HD97778", "HD108907", "HD119850", "HD121130", "HD129712", "GJ687", "HD168574", "HD173739", "HD176124", "HD187082", "HD28484", "HD47548", "2MASSJ18150712-0018523", "GJ876", "HD224062", "HD58061", "HD133216", "HD171394", "HD173740", "HD19058", "HD25705", "HD8019", "GJ699", "HD172171", "HD172816", "HD175588", "IRC-20356", "HD18310", "HD39983", "HD132813", "HD132112", "HD136695", "HD150383", "HD156014", "HIP87668", "HD175865", "HD246450", "HD149880", "HD1760", "GJ551", "HD152276", "HD19737", "HD28236", "HD7773", "HD18191", "HD70421", "HD78420", "HD78712", "HD148783", "HD189124", "HD36231", "HD218997", "HD12025", "HD200994", "HD117833", "HD141850", "HD145459", "IRC+20128", "HD39741", "HD14386", "HD219946", "HD17446", "HD36090", "HD114961", "HD139492", "IRC+00269", "HIP96647", "IRC+10498", "HD218292", "HD220033", "HD136753", "HD148206", "HD185293", "HD202012", "HD222800", "HD34019", "HD117287", "HD224709", "HD188915", "HD39816", "HD69243", "HD84346", "HD120499", "HD177940", "HD1845", "HD29712", "HD84748", "HD126753", "HD258476", "HD120285"]


import astropy.io.votable
# create votable from data table and store it in the temporary file
jmdc_table = Table()
jmdc_table.add_column(jmdc_ids, name="id")
jmdc_vot = astropy.io.votable.from_table(jmdc_table)
jmdc_vot_filename = "jmdc.vot"
jmdc_vot.to_xml(jmdc_vot_filename)


#### Search for coordinates counter part in Simbad

In [9]:
#"SELECT oid AS id, ra, dec, main_id FROM basic JOIN ident ON oidref=oid WHERE id = '

query = """SELECT jmdc.id, ident.id AS "SimbadId", basic.ra, basic.dec, basic.otype_txt 
FROM TAP_UPLOAD.jmdc INNER JOIN ident ON ident.id = jmdc.id INNER JOIN basic ON ident.oidref = basic.oid"""

jmdc_withcoo = simbad.search(query, uploads={'jmdc':jmdc_vot_filename} )
jmdc_withcoo_filename="jmdc_coo.vot"
jmdc_withcoo.votable.to_xml(jmdc_withcoo_filename)



#### Search for public data on OiDB related to JMDC's stars fouseRemoteJmcsNamesach calibrated levels

In [10]:
oidb_records={}
for calib_level in range(1,4):
    query = f"""SELECT DISTINCT jmdc.id AS "jmdcId", oidb.access_url 
FROM TAP_UPLOAD.jmdc LEFT OUTER JOIN oidb 
ON ( CONTAINS(POINT('ICRS', oidb.s_ra, oidb.s_dec), CIRCLE('ICRS', jmdc.ra, jmdc.dec, 0.033))=1 )
WHERE oidb.calib_level={calib_level} AND oidb.nb_vis2>1 AND ( oidb.data_rights='public' OR oidb.obs_release_date < '{date.today().isoformat()}T00:00:00Z' )
"""
    oidb_records[calib_level] = oidb.search(query, uploads={'jmdc':jmdc_withcoo_filename} )

### Show summary infos and send urls as oixp collection to OIFitExplorer

In [11]:
print(f"{len(jmdc_vot.get_first_table().to_table())} targets names on JMDC")
print(f"{len(jmdc_withcoo)} targets resolved by Simbad")
for calib_level in oidb_records:
    print(f"{len(oidb_records[calib_level])} OIFits (L{calib_level}) records found on OiDB")

604 targets names on JMDC
604 targets resolved by Simbad
20 OIFits (L1) records found on OiDB
1440 OIFits (L2) records found on OiDB
127 OIFits (L3) records found on OiDB


In [ ]:
for calib_level in oidb_records:
    input(f"Press a key to send data for calib_level={calib_level} ...")
    sendAsOixp(oidb_records[calib_level]["access_url"])